# Случайные процессы. Прикладной поток.
## Практическое задание 6

**Правила:**

* Выполненную работу нужно отправить на почту `probability.diht@yandex.ru`, указав тему письма `"[СП17] Фамилия Имя - Задание 6"`. Квадратные скобки обязательны. Вместо `Фамилия Имя` нужно подставить свои фамилию и имя.
* Прислать нужно ноутбук и его pdf-версию. Названия файлов должны быть такими: `6.N.ipynb` и `6.N.pdf`, где `N` - ваш номер из таблицы с оценками.
* При проверке могут быть запущены функции, которые отвечают за генерацию траекторий винеровского процесса.

------------------

In [11]:
import scipy.stats as sps
import numpy as np

### 1. Генерация винеровского процесса

Генерировать траектории винеровского процесса можно двумя способами:

1. На отрезке $[0, 1]$ траектория генерируется с помощью функций Шаудера. Описание данного метода было рассказано на лекции. Его можно найти так же в книге *А.В. Булинский, А.Н. Ширяев - Теория случайных процессов*.

2. На отрезке $[0, \pi]$ траекторию можно с помощью следующей формулы
$$W_t = \frac{\xi_0 t}{\sqrt{\pi}} +\sqrt{\frac{2}{\pi}}\sum\limits_{i=1}^{+\infty} \frac{\sin(kt)}{k} \xi_k,$$
где $\{\xi_n\}$ --- независимые стандартные нормальные случайные величины.

Траектория для $\mathbb{R}_+$ генерируется с помощью генерирования отдельных траекторий для отрезков длины 1 или $\pi$ (в зависимости от метода) с последующим непрерывным склеиванием.

Генерацию траекторий одним из этих способов вам нужно реализовать. Ваш вариант можете узнать из файла с распределением.

Напишите класс, который будет моделировать винеровский процесс.
Из бесконечной суммы берите первые $n$ слагаемых, где число $n$ соответствует параметру `precision`.
Интерфейс должен быть примерно таким (подчеркивания обязательны!):

In [401]:
class WinerProcess:
    def __init__(self, precision=10000):
        self.precision = precision
        self.xi = [sps.norm.rvs(size=precision)]
        self.n = np.log2(np.arange(1, precision)).astype(int)
        self.a_nk = (1 / np.power(2, self.n)) * (np.arange(1, precision) - np.power(2, self.n))
        
    def __getitem__(self, times):
        last_time = np.max(times)
        if (last_time - int(last_time) > 0):
            last_time = int(last_time) + 1
        else:
            last_time = last_time.astype(int)
        if (last_time > len(self.xi)) :
            self.xi += [sps.norm.rvs(size=self.precision)] * (last_time - len(self.xi))
        for i in range(last_time):
            S = np.zeros(self.precision - 1)
            # self.a_nk[::2] <+ times[i] and 
            print(self.a_nk[:10])
            print((self.a_nk + (1 / np.power(2, self.n + 1)))[:10])
            positive_slope = np.logical_and(
                (times[i] <= self.a_nk + (1 / np.power(2, self.n + 1))),
                (self.a_nk <= times[i]))
            print(positive_slope[:10])
            S[positive_slope] = (times[i] - self.a_nk[positive_slope]) /\
                                (np.power(2, self.n[positive_slope] / 2 + 1))
            #negative_slope = np.logical_and(*something*)
            #S[negative_slope] = *something*
            #Need to distinguish the first element of the positive_slope
            #Also need to add zero Haar function
            print(S[:10])

In [402]:
p = WinerProcess()
t = np.linspace(0.1, 0.1, 1)
p[t]

[ 0.     0.     0.5    0.     0.25   0.5    0.75   0.     0.125  0.25 ]
[ 0.5     0.25    0.75    0.125   0.375   0.625   0.875   0.0625  0.1875
  0.3125]
[ True  True False  True False False False False False False]
[ 0.05        0.03535534  0.          0.025       0.          0.          0.
  0.          0.          0.        ]


In [273]:
x = np.arange(1, 100)
x[1::2]

array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34,
       36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68,
       70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98])

* Экземпляр класса должен представлять некоторую траекторию винеровского процесса. Это означает, что один и тот же экземпляр класса для одного и того же момента времени должен возвращать одно и тоже значение. Разные экземпляры класса --- разные (п.н.) траектории.

* Метод `__init__` (конструктор) должен запоминать число слагаемых в сумме (`precision`), а также (может быть) генерировать необходимые случайные величины для начального отрезка.

* Метод `__getitem__` должен принимать набор моментов времени и возвращать значения траектории винеровского процесса в эти моменты времени. При необходимости можно сгенерировать новые случайные величины. Используйте то, что запись `x.__getitem__(y)` эквивалентна `x[y]`.

* Для получения полного балла и быстро работающего кода реализация должна содержать не более одного явного цикла (по отрезкам при непосредственной генерации). Вместо всех остальных циклов нужно использовать функции библиотеки `numpy`.

* Внимательно проверьте отсутствие разрывов траектории в точках, кратных $\pi$.

* Имена любых вспомогательных методов должны начинаться с одного подчеркивания.

* В реализации желательно комментировать (почти) каждую строку кода. Это даже больше поможет вам, чем нам при проверке.

Сгенерируйте траекторию винеровского процесса и постройте ее график. Сгенерируйте еще одну траекторию и постройте график двумерного винеровского процесса. Графики должны быть похожими на графики с семинара.

In [ ]:
...

Допустим, для исследования свойств траекторий винеровского процесса нам нужно сгенерировать траекторию с хорошей точностью до достаточно большого значения $t$.
Какие проблемы могут возникнуть при использовании реализованного класса?
Для этого попробуйте запустить следующий код.

In [ ]:
Wt = WinerProccess()
t = np.linspace(0, 10 ** 7, 10 ** 5)
values = Wt[t]

Опишите подробно причину проблемы, которая в данном случае возникает.

...

Для избавления от таких проблем реализуйте следующую функцию:

In [ ]:
def winer_proccess_path(end_time, step, precision=10000):
    # Моменты времени, в которые нужно вычислить значения
    times = np.arange(0, end_time, step)
    # Сюда запишите значения траектории в моменты времени times
    values = np.zeros_like(times)
    ...

    return times, values

Для получения полного балла и быстро работающего кода реализация должна содержать не более одного явного цикла (по отрезкам при непосредственной генерации). Вместо всех остальных циклов нужно использовать функции библиотеки `numpy`. Внутри этой функции можно реализовать вспомогательную функцию.

### 2. Исследования

**Следующая часть работы делается в паре.**

Для каждого их двух способов генерация траектрии винеровского процесса постройте таблицу $3 \times 3$ из графиков траекторий винеровского процесса.
По вертикали изменяйте количество $n$ используемых слагаемых в сумме ($n=10; 100; 1000$), по горизонтали --- длину отрезка, на котором генерируется винеровский процесс (использовать отрезки $[0, 10], [0, 1], [0, 0.1]$).
Обратите внимание, что от размера сетки зависит только точность отображения функции на графике, а не сама функция, поэтому сетку нужно выбирать достаточно мелкой.

In [ ]:
plt.figure(figsize=(20, 12))
for i, precision in enumerate([10, 100, 1000]):
    for j, max_time in enumerate([10, 1, 0.1]):
        t = np.linspace(0, max_time, 1000)
        values = WinerProccess(precision=precision)[t]
        
        plt.subplot(3, 3, i * 3 + j + 1)
        plt.plot(t, values)
        plt.title('precision = %d' % precision)
plt.show()

Какие выводы можно сделать про каждый способ генерации?

...

Сравните два способа генерации по времени работы.

In [ ]:
%time times, values = winer_proccess_path(100000, 0.1)

Постройте графики полученных траекторий для каждого способа? Отличаются ли траектории визуально?

Какие можно сделать выводы из сравнения двух способов генерации?

**Следующая часть работы делается индивидуально.**

1. Сгенерируйте 100 траекторий винеровского процесса с достаточно хорошей точностью и нарисуйте их на одном графике? Что можно сказать про поведение траекторий?

2. Нарисуйте график двумерного винеровского процесса (см. презентацию с семинара).